In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

In [3]:
Nx = 200
Co = 0.2
tMax = 1 # time: 0 - 1
viscosity_coeff = (0.01) / np. pi
x = np.linspace(-1,1, Nx)
u0 = -np.sin(np.pi)
